## (E4)Sentimental_Analysis_of_Movie_Dataset_from_NAVER_InYu
### 순서

### 1) 데이터 준비와 확인

2) 데이터로더 구성

3) 모델구성을 위한 데이터 분석 및 가공

    데이터셋 내 문장 길이 분포
    적절한 최대 문장 길이 지정
    keras.preprocessing.sequence.pad_sequences 을 활용한 패딩 추가

4) 모델구성 및 validation set 구성

   모델은 3가지 이상 다양하게 구성하여 실험해 보세요.
   
5) 모델 훈련 개시

6) Loss, Accuracy 그래프 시각화

7) 학습된 Embedding 레이어 분석 - 유사도 단어
    
8) 한국어 Word2Vec 임베딩 활용하여 성능개선


In [1]:
# 필요한 모듈 import 와 read data
import pandas as pd
import urllib.request
%matplotlib inline
import matplotlib.pyplot as plt
import re
from konlpy.tag import Okt
from tensorflow import keras
from tensorflow.keras.preprocessing.text import Tokenizer
import numpy as np
from tensorflow.keras.preprocessing.sequence import pad_sequences
from collections import Counter
from konlpy.tag import Mecab
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

# read data
train_data = pd.read_table('~/aiffel/sentiment_classification/ratings_train.txt')
test_data = pd.read_table('~/aiffel/sentiment_classification/ratings_test.txt')

# data 개수
print('훈련용 data 개수 :', len(train_data))
print('테스트용 data 개수 :', len(test_data))

훈련용 data 개수 : 150000
테스트용 data 개수 : 50000


In [2]:
# {2} 데이터 확인해보기
train_data[:5]

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1


부정 감상평 label은 0  
긍정 감상평 lable은 1

# 비중복 data 개수
```train_data['document'].nunique(), train_data['label'].nunique()```

label은 0, 1뿐이라서 당연히 2개지만 150000개에서 약 4000개 정도의 중복 리뷰가 있다는 것이다.  
중복된 리뷰를 삭제한다.

# 중복 리뷰 삭제
```train_data.drop_duplicates(subset=['document'], inplace=True)```

train_data에서 해당 리뷰의 긍, 부정 유무가 기재되어있는 레이블(label) 값의 분포를 가시적으로 확인

```train_data['label'].value_counts().plot(kind = 'bar')
print(train_data.groupby('label').size().reset_index(name = 'count'))```

부정 감상평이 살짝 많지만 근사한 data 개수를 가진 것을 확일할 수 있었다.  
감상평 중에 Null 값을 가진 샘플이 있는지 확인해보자.

```print(train_data.isnull().values.any()) # True는 Null 값을 가진 샘플 존재한다는 뜻
print(train_data.isnull().sum())
train_data.loc[train_data.document.isnull()] # 목록에서 위치 확인```

# Null 샘플  제거
```train_data = train_data.dropna(how = 'any') # Null 값이 존재하는 행 제거
print(train_data.isnull().values.any()) # False가 나와야 Null 샘플이 없다.
print(len(train_data)) # Null 값을 제외한 학습 data```

# 한글과 공백을 제외하고 모두 제거
```
train_data['document'] = train_data['document'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")```

# 다시 Null값 확인하고 Null 제거
```train_data['document'].replace('', np.nan, inplace=True)
train_data = train_data.dropna(how = 'any')
print('전처리 후 학습용 샘플의 개수 :',len(train_data))```

# 나머지 test data도 동일하게 진행
```test_data.drop_duplicates(subset = ['document'], inplace=True) # document 열에서 중복인 내용이 있다면 중복 제거
test_data['document'] = test_data['document'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","") # 정규 표현식 수행
test_data['document'].replace('', np.nan, inplace=True) # 공백은 Null 값으로 변경
test_data = test_data.dropna(how='any') # Null 값 제거
print('전처리 후 테스트용 샘플의 개수 :',len(test_data))```

# 불용어 제거
```stopwords = ['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다']
okt = Okt()
okt.morphs('이 영화 진짜 인생영화임', stem = True)```

Okt는 열심히 다운받았던 KoNLPy에서 제공하는 형태소 분석기이다. 한국어 토큰화는 띄어쓰기 기준이 아닌 형태소 분석기를 사용한다.  
토큰화 하면서 불용어를 제거하여 x_train에 저장

```
tokenizer = Mecab()
x_train = []
for sentence in train_data['document']:
    temp_x = tokenizer.morphs(sentence) # 토큰화
    temp_x = [word for word in temp_x if not word in stopwords] # 불용어 제거
    x_train.append(temp_x)
    
x_test = []
for sentence in test_data['document']:
    temp_x = tokenizer.morphs(sentence) # 토큰화
    temp_x = [word for word in temp_x if not word in stopwords] # 불용어 제거
    x_test.append(temp_x)
```

x_train에서 빈도가 높은 순으로 정수를 부여받는다. 그래서 이중에서 가장 빈도 수가 많은 9996개의 단어로 리스트를만들어서 정수로 인코딩해준다.  
앞에는 \<BOS>, \<PAD>, \<UNK>, \<UNUSED>는 관례적으로 딕셔너리 맨 앞에 넣어줍니다. 

# 정수 인코딩 
```words = np.concatenate(x_train).tolist()
counter = Counter(words)
counter = counter.most_common(10000-4) # 변경가능
vocab = ['<PAD>', '<BOS>', '<UNK>', '<UNUSED>'] + [key for key, _ in counter]
word_to_index = {word:index for index, word in enumerate(vocab)}
index_to_word = {index:word for word, index in word_to_index.items()}```

### 2) 데이터로더 구성
전처리를 모두 끝낸 데이터를 x_train, y_train, x_test, y_test에 각각 load한다.

data에 있는 단어 중 9996개에 들지 못한 단어는 <UNK>으로 변경하고 words에 포함된 단어라면 인코딩을 진행합니다.

In [3]:
from konlpy.tag import Mecab
tokenizer = Mecab()
stopwords = ['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다']

def load_data(train_data, test_data, num_words=10000):
    train_data.drop_duplicates(subset=['document'], inplace=True)
    train_data = train_data.dropna(how = 'any') 
    test_data.drop_duplicates(subset=['document'], inplace=True) 
    test_data = test_data.dropna(how = 'any') 

    x_train = []
    for sentence in train_data['document']:
        temp_x = tokenizer.morphs(sentence) # 토큰화
        temp_x = [word for word in temp_x if not word in stopwords] # 불용어 제거
        x_train.append(temp_x)

    x_test = []
    for sentence in test_data['document']:
        temp_x = tokenizer.morphs(sentence) # 토큰화
        temp_x = [word for word in temp_x if not word in stopwords] # 불용어 제거
        x_test.append(temp_x)

    words = np.concatenate(x_train).tolist()
    counter = Counter(words)
    counter = counter.most_common(10000-4)
    vocab = ['<PAD>', '<BOS>', '<UNK>', '<UNUSED>'] + [key for key, _ in counter]
    word_to_index = {word:index for index, word in enumerate(vocab)} # enumerate 열거하다
#     index_to_word = {index:word for word, index in word_to_index.items()}

    def wordlist_to_indexlist(wordlist):
        return [word_to_index[word] if word in word_to_index else word_to_index['<UNK>'] for word in wordlist]

    x_train = list(map(wordlist_to_indexlist, x_train))
    x_test = list(map(wordlist_to_indexlist, x_test))

    return x_train, np.array(list(train_data['label'])), x_test, np.array(list(test_data['label'])), word_to_index

x_train, y_train, x_test, y_test, word_to_index = load_data(train_data, test_data)

In [4]:
# def wordlist_to_indexlist(wordlist):
#     return [word_to_index[word] if word in word_to_index else word_to_index['<UNK>'] for word in wordlist]

# x_train = list(map(wordlist_to_indexlist, x_train))
# x_test = list(map(wordlist_to_indexlist, x_test))

# # y는 별도로 저장
# y_train = np.array(train_data['label'])
# y_test = np.array(test_data['label'])

# print(len(x_train))
# print(len(x_test))

### 3) 모델구성을 위한 데이터 분석 및 가공
문장 길이를 평준화해서 벡터의 길이는 조정한다.  
Embedding 레이어의 인풋이 되는 문장 벡터는 그 길이가 일정해야 하기 때문이다.  
긴 문장은 자르고 짧은 단어는 \<PAD>를 패딩해서 길이를 맞춰준다.

In [5]:
total_data_text = list(x_train) + list(x_test)

# 데이터셋 내 문장 길이 분포 확인
num_tokens = [len(tokens) for tokens in total_data_text]
num_tokens = np.array(num_tokens)

# 문장길이의 평균값, 최대값, 표준편차를 계산
print('문장길이 평균 : ', np.mean(num_tokens))
print('문장길이 최대 : ', np.max(num_tokens))
print('문장길이 표준편차 : ', np.std(num_tokens))

# 위의 연산값을 이용해서 전체 문장 95% 정도를 포함할 수 있게 적절한 최대 문장 길이 지정
max_tokens = np.mean(num_tokens) + round(2.5 * np.std(num_tokens)) 
maxlen = int(max_tokens)
print('pad_sequences maxlen : ', maxlen)
print('전체 문장의 {}%가 maxlen 설정값 이내에 포함됩니다. '.format(np.sum(num_tokens < max_tokens)*100 / len(num_tokens)))

문장길이 평균 :  15.969376315021577
문장길이 최대 :  116
문장길이 표준편차 :  12.843535456326455
pad_sequences maxlen :  47
전체 문장의 95.2175448835102%가 maxlen 설정값 이내에 포함됩니다. 


RNN은 입력데이터가 순차적으로 처리되기 때문에 가장 마지막 입력이 최종 state 값에 가장 영향을 많이 미치게 됩니다. 그러므로 마지막 입력이 무의미한 padding으로 채워지는 것은 비효율적입니다. 따라서 'pre'가 훨씬 유리하며, 10% 이상의 테스트 성능 차이를 보이게 됩니다.

In [6]:
# 위의 maxlen 값에 맞춰서 패딩
# post는 data 뒤에 패딩, pre는 data 앞에 패딩
# keras.preprocessing.sequence.pad_sequences 을 활용한 패딩 추가
x_train = keras.preprocessing.sequence.pad_sequences(x_train,
                                                       value=word_to_index["<PAD>"],
                                                       padding='pre',
                                                       maxlen=maxlen)

x_test = keras.preprocessing.sequence.pad_sequences(x_test,
                                                      value=word_to_index["<PAD>"],
                                                      padding='pre',
                                                      maxlen=maxlen)

### 4) 모델구성 및 validation set 구성

In [7]:
from tensorflow.keras.layers import Embedding, Dense, LSTM
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

vocab_size = 10000    # 어휘 사전의 크기입니다(10,000개의 단어)
word_vector_dim = 16  # 워드 벡터의 차원수 (변경가능한 하이퍼파라미터)

#Conv1D
model_CNN = keras.Sequential()
model_CNN.add(keras.layers.Embedding(vocab_size, word_vector_dim, input_shape=(None,)))
model_CNN.add(keras.layers.Conv1D(16, 7, activation='relu'))
model_CNN.add(keras.layers.MaxPooling1D(5))
model_CNN.add(keras.layers.Conv1D(16, 7, activation='relu'))
model_CNN.add(keras.layers.GlobalMaxPooling1D())
model_CNN.add(keras.layers.Dense(8, activation='relu'))
model_CNN.add(keras.layers.Dense(1, activation='sigmoid'))  # 최종 출력은 긍정/부정을 나타내는 1dim 입니다.


#GlobalMaxPooling1D() 레이어 하나만 사용하는 방법
#전체 문장 중에서 단 하나의 가장 중요한 단어만 피처로 추출하여 그것으로 문장의 긍정/부정을 평가하는 방식
model_GlobMP = keras.Sequential()
model_GlobMP.add(keras.layers.Embedding(vocab_size, word_vector_dim, input_shape=(None,)))
model_GlobMP.add(keras.layers.GlobalMaxPooling1D())
#model_GlobMP.add(keras.layers.Dropout(0.3))
model_GlobMP.add(keras.layers.Dense(8, activation='relu'))
#model_GlobMP.add(keras.layers.Dropout(0.3))
model_GlobMP.add(keras.layers.Dense(1, activation='sigmoid'))  # 최종 출력은 긍정/부정을 나타내는 1dim 입니다.


# RNN
model_RNN = keras.Sequential()
model_RNN.add(keras.layers.Embedding(vocab_size, word_vector_dim, input_shape=(None,))) # 동일 # model의 첫번째 레이어
model_RNN.add(keras.layers.Conv1D(16, 7, activation='relu'))
model_RNN.add(keras.layers.MaxPooling1D(5))
model_RNN.add(keras.layers.Conv1D(16, 7, activation='relu'))
model_RNN.add(keras.layers.GlobalMaxPooling1D())
model_RNN.add(keras.layers.Dense(8, activation='relu')) # 동일
model_RNN.add(keras.layers.Dense(1, activation='sigmoid'))  # 최종 출력은 긍정/부정을 나타내는 1dim 입니다. # 동일


#LSTM
model_LSTM = keras.Sequential()
model_LSTM.add(keras.layers.Embedding(vocab_size, word_vector_dim, input_shape=(None,)))
model_LSTM.add(keras.layers.SpatialDropout1D(0.4))
model_LSTM.add(keras.layers.LSTM(word_vector_dim, dropout=0.2, recurrent_dropout=0.2))   # 가장 널리 쓰이는 RNN인 LSTM 레이어를 사용하였습니다.
model_LSTM.add(keras.layers.Dense(1, activation='sigmoid'))  # 최종 출력은 긍정/부정을 나타내는 1dim 입니다.

# model = Sequential()
# model.add(Embedding(vocab_size, 100))
# model.add(LSTM(128))
# model.add(Dense(1, activation='sigmoid'))

model_CNN.summary()
model_GlobMP.summary()
model_RNN.summary()
model_LSTM.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 16)          160000    
_________________________________________________________________
conv1d (Conv1D)              (None, None, 16)          1808      
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, None, 16)          0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, None, 16)          1808      
_________________________________________________________________
global_max_pooling1d (Global (None, 16)                0         
_________________________________________________________________
dense (Dense)                (None, 8)                 136       
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 9

In [8]:
# validation set 10000건 분리
x_val = x_train[:10000]   
y_val = y_train[:10000]

# validation set을 제외한 나머지 15000건
partial_x_train = x_train[10000:]  
partial_y_train = y_train[10000:]

print(x_train.shape)
print(y_train.shape)
print(partial_x_train.shape)
print(partial_y_train.shape)

(146182, 47)
(146182,)
(136182, 47)
(136182,)


### 5) 모델 훈련 개시
#### 1. 1-D CNN

In [9]:
# model 학습 시작 "compile"
model_CNN.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])
              
epochs=10  # 몇 epoch를 훈련하면 좋을지 결과를 보면서 바꾸어 봅시다. 

history_model_CNN= model_CNN.fit(partial_x_train,
                    partial_y_train,
                    epochs=epochs,
                    batch_size=512,
                    validation_data=(x_val, y_val),
                    verbose=1)
                           
# 테스트셋을 통한 모델 평가
results = model_CNN.evaluate(x_test,  y_test, verbose=2)
print(results)

Epoch 1/10
266/266 [==============================] - 9s 33ms/step - loss: 0.5095 - accuracy: 0.7398 - val_loss: 0.3629 - val_accuracy: 0.8363
Epoch 2/10
266/266 [==============================] - 1s 5ms/step - loss: 0.3422 - accuracy: 0.8503 - val_loss: 0.3521 - val_accuracy: 0.8443
Epoch 3/10
266/266 [==============================] - 1s 5ms/step - loss: 0.3079 - accuracy: 0.8691 - val_loss: 0.3476 - val_accuracy: 0.8482
Epoch 4/10
266/266 [==============================] - 1s 5ms/step - loss: 0.2799 - accuracy: 0.8833 - val_loss: 0.3525 - val_accuracy: 0.8469
Epoch 5/10
266/266 [==============================] - 1s 5ms/step - loss: 0.2506 - accuracy: 0.8990 - val_loss: 0.3619 - val_accuracy: 0.8442
Epoch 6/10
266/266 [==============================] - 1s 5ms/step - loss: 0.2202 - accuracy: 0.9132 - val_loss: 0.3838 - val_accuracy: 0.8421
Epoch 7/10
266/266 [==============================] - 1s 5ms/step - loss: 0.1901 - accuracy: 0.9280 - val_loss: 0.4111 - val_accuracy: 0.8392
Epoch

#### 2. GlobMP

In [15]:
# model 학습 시작 "compile"
model_GlobMP.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])
              
epochs=10  # 몇 epoch를 훈련하면 좋을지 결과를 보면서 바꾸어 봅시다. 

history_model_GlobMP= model_CNN.fit(partial_x_train,
                    partial_y_train,
                    epochs=epochs,
                    batch_size=512,
                    validation_data=(x_val, y_val),
                    verbose=1)
                           
# 테스트셋을 통한 모델 평가
results_GlobMP = model_GlobMP.evaluate(x_test,  y_test, verbose=2)
print(results_GlobMP)

Epoch 1/10
266/266 [==============================] - 1s 4ms/step - loss: 0.0387 - accuracy: 0.9860 - val_loss: 1.3454 - val_accuracy: 0.8050
Epoch 2/10
266/266 [==============================] - 1s 4ms/step - loss: 0.0327 - accuracy: 0.9886 - val_loss: 1.3977 - val_accuracy: 0.8045
Epoch 3/10
266/266 [==============================] - 1s 5ms/step - loss: 0.0310 - accuracy: 0.9893 - val_loss: 1.4531 - val_accuracy: 0.8034
Epoch 4/10
266/266 [==============================] - 1s 5ms/step - loss: 0.0300 - accuracy: 0.9897 - val_loss: 1.5381 - val_accuracy: 0.8025
Epoch 5/10
266/266 [==============================] - 1s 5ms/step - loss: 0.0291 - accuracy: 0.9900 - val_loss: 1.5738 - val_accuracy: 0.8013
Epoch 6/10
266/266 [==============================] - 1s 4ms/step - loss: 0.0285 - accuracy: 0.9901 - val_loss: 1.6338 - val_accuracy: 0.8004
Epoch 7/10
266/266 [==============================] - 1s 4ms/step - loss: 0.0279 - accuracy: 0.9902 - val_loss: 1.6973 - val_accuracy: 0.8007
Epoch 

ValueError: Failed to find data adapter that can handle input: (<class 'list'> containing values of types {"(<class 'list'> containing values of types set())", '(<class \'list\'> containing values of types {"<class \'int\'>"})'}), <class 'numpy.ndarray'>

### 3. RNN

In [11]:
# model 학습 시작 "compile"
model_RNN.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])
              
epochs=10  # 몇 epoch를 훈련하면 좋을지 결과를 보면서 바꾸어 봅시다. 

history_model_RNN= model_CNN.fit(partial_x_train,
                    partial_y_train,
                    epochs=epochs,
                    batch_size=512,
                    validation_data=(x_val, y_val),
                    verbose=1)
                           
# 테스트셋을 통한 모델 평가
results_RNN = model_RNN.evaluate(x_test,  y_test, verbose=2)
print(results_RNN)

Epoch 1/10
266/266 [==============================] - 1s 4ms/step - loss: 0.0531 - accuracy: 0.9825 - val_loss: 1.1104 - val_accuracy: 0.8115
Epoch 2/10
266/266 [==============================] - 1s 4ms/step - loss: 0.0517 - accuracy: 0.9824 - val_loss: 1.1105 - val_accuracy: 0.8101
Epoch 3/10
266/266 [==============================] - 1s 4ms/step - loss: 0.0519 - accuracy: 0.9819 - val_loss: 1.1965 - val_accuracy: 0.8099
Epoch 4/10
266/266 [==============================] - 1s 4ms/step - loss: 0.0494 - accuracy: 0.9828 - val_loss: 1.2700 - val_accuracy: 0.8083
Epoch 5/10
266/266 [==============================] - 1s 4ms/step - loss: 0.0478 - accuracy: 0.9833 - val_loss: 1.2767 - val_accuracy: 0.8059
Epoch 6/10
266/266 [==============================] - 1s 4ms/step - loss: 0.0445 - accuracy: 0.9851 - val_loss: 1.2911 - val_accuracy: 0.8096
Epoch 7/10
266/266 [==============================] - 1s 4ms/step - loss: 0.0425 - accuracy: 0.9854 - val_loss: 1.3781 - val_accuracy: 0.8076
Epoch 

### 4. LSTM

In [12]:
# model 학습 시작 "compile"
model_LSTM.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])
              
epochs=10  # 몇 epoch를 훈련하면 좋을지 결과를 보면서 바꾸어 봅시다. 

history_model_LSTM= model_CNN.fit(partial_x_train,
                    partial_y_train,
                    epochs=epochs,
                    batch_size=512,
                    validation_data=(x_val, y_val),
                    verbose=1)
                           
# 테스트셋을 통한 모델 평가
results_LSTM = model_LSTM.evaluate(x_test,  y_test, verbose=2)
print(results_LSTM)

Epoch 1/10
266/266 [==============================] - 1s 4ms/step - loss: 0.0415 - accuracy: 0.9856 - val_loss: 1.4921 - val_accuracy: 0.8069
Epoch 2/10
266/266 [==============================] - 1s 4ms/step - loss: 0.0399 - accuracy: 0.9859 - val_loss: 1.4635 - val_accuracy: 0.8051
Epoch 3/10
266/266 [==============================] - 1s 4ms/step - loss: 0.0392 - accuracy: 0.9863 - val_loss: 1.5097 - val_accuracy: 0.7975
Epoch 4/10
266/266 [==============================] - 1s 4ms/step - loss: 0.0387 - accuracy: 0.9867 - val_loss: 1.5622 - val_accuracy: 0.8011
Epoch 5/10
266/266 [==============================] - 1s 4ms/step - loss: 0.0386 - accuracy: 0.9865 - val_loss: 1.5974 - val_accuracy: 0.8015
Epoch 6/10
266/266 [==============================] - 1s 4ms/step - loss: 0.0389 - accuracy: 0.9864 - val_loss: 1.6254 - val_accuracy: 0.8032
Epoch 7/10
266/266 [==============================] - 1s 4ms/step - loss: 0.0384 - accuracy: 0.9866 - val_loss: 1.6564 - val_accuracy: 0.7990
Epoch 

In [13]:
# 시간오래걸린다
from konlpy.tag import Mecab
tokenizer = Mecab()
stopwords = ['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다']

def load_data(train_data, test_data, num_words=10000):
    train_data.drop_duplicates(subset=['document'], inplace=True)
    train_data = train_data.dropna(how = 'any') 
    test_data.drop_duplicates(subset=['document'], inplace=True) # 어떤 비율로 train과 test를 나눴는지 모르겠음
    test_data = test_data.dropna(how = 'any') 

    x_train = []
    for sentence in train_data['document']:
        temp_x = tokenizer.morphs(sentence) # 토큰화
        temp_x = [word for word in temp_x if not word in stopwords] # 불용어 제거
        x_train.append(temp_x)

    x_test = []
    for sentence in test_data['document']:
        temp_x = tokenizer.morphs(sentence) # 토큰화
        temp_x = [word for word in temp_x if not word in stopwords] # 불용어 제거
        x_test.append(temp_x)

    words = np.concatenate(x_train).tolist()
    counter = Counter(words)
    counter = counter.most_common(10000-4)
    vocab = ['<PAD>', '<BOS>', '<UNK>', '<UNUSED>'] + [key for key, _ in counter]
    word_to_index = {word:index for index, word in enumerate(vocab)} # enumerate 열거하다
#     index_to_word = {index:word for word, index in word_to_index.items()}

    def wordlist_to_indexlist(wordlist):
        return [word_to_index[word] if word in word_to_index else word_to_index['<UNK>'] for word in wordlist]

    x_train = list(map(wordlist_to_indexlist, x_train))
    x_test = list(map(wordlist_to_indexlist, x_test))

    return x_train, np.array(list(train_data['label'])), x_test, np.array(list(test_data['label'])), word_to_index

x_train, y_train, x_test, y_test, word_to_index = load_data(train_data, test_data)